In [ ]:
# --------------------------------------------------------------#
#                   Preparamos el Dataset                       #
#---------------------------------------------------------------#
import pandas as pd
from datasets import Dataset, ClassLabel, DatasetDict, Sequence
from dataset import datos

#train_csv = "./train.csv"
tags = ['O', 'B-DSEM', 'I-DSEM','B-HORA', 'I-HORA', 'B-MIN', 'I-MIN', 'B-RELH', 'I-RELH', 'B-RELM', 'I-RELM', 'B-RELS', 'I-RELS', 'B-MTN', 'I-MTN', 'B-TIT', "I-TIT"]
"""Tags para las entidades a reconocer

    Elementos
    ----------
        
        * 'O': no es entidad
        * 'DSEM': día de la semana
        * 'HORA': hora del día
        * 'MIN': minutos dentro de la hora
        * 'REL': tiempo relativo a la hora actual (una hora, 30 minutos, etc.)
        * 'MTN': Mañana/Tarde/Noche, si no se especifica por defecto será PM para las [1:00->7:00) y AM para las [7:00->12:00] 
        * 'TIT': título
"""

id2tag = {id: tag for id, tag in enumerate(tags)}
'''Diccionario de IDs a sus correspondientes etiquetas de entidades'''

tag2id = {tag: id for id, tag in enumerate(tags)}
'''Diccionario de etiquetas de entidades a sus correspondientes IDs'''

print('IMPORTANDO LOS DATASETS...')

# Leemos el fichero con los datos de entrenamiento y testeo
#df = pd.read_csv(train_csv)
df = pd.DataFrame()

# Transformamos los datos del dataset para obtener las columnas
# que nos interesan. En este caso serían los comandos separados
# por palabras (tokens), las etiquetas de las entidades en formato
# numérico (ner_tags) y en cadena de texto (ner_tags_str)

# DESDE EL FICHERO dataset.py:
df['tokens'] = datos['comandos']
df['ner_tags_str'] = datos['tokens']

df['ner_tags'] = df['ner_tags_str'].map(lambda lista: list(map(lambda tag: tag2id[tag], lista)))


# Construímos el Dataset a partir de las 3 columnas anteriores y convertimos la columna 
# de ner_tags a un ClassLabel para mayor facilidad para el procesamiento
labels=ClassLabel(names=tags)
data = Dataset.from_pandas(df[['tokens','ner_tags_str','ner_tags']])
data = data.cast_column("ner_tags", Sequence(feature=labels))

# Separamos el Dataset en dos distintos; uno para entrenamiento y otro para testeo,
# y los agrupamos en un diccionario de Datasets
train_dataset, test_dataset = data.train_test_split(test_size=0.01, shuffle=False).values()
data = DatasetDict({'train': train_dataset, 'test': test_dataset})

In [ ]:
# --------------------------------------------------------------#
#                           Tokenizamos                         #
#---------------------------------------------------------------#
from transformers import AutoTokenizer

print('TOKENIZANDO INPUTS...')

nombre_modelo = 'dccuchile/distilbert-base-spanish-uncased'
tokenizer = AutoTokenizer.from_pretrained(nombre_modelo)

def alinear(ids_palabras_tokenizadas, ner_tags_originales):
    """
        Alinea los ner_tags originales, es decir las etiquetas BIO numeradas correspondientes
        a los tokens originales, con los nuevos tokens resultantes de tokenizar las palabras originales
        mediante el modelo preentrenado.

        Devuelve
        --------

        new_tags
            Nueva lista de ner_tags alineada con los nuevos tokens.
    """

    new_tags = list(
        map(
            lambda x: -100 if x == None else ner_tags_originales[x],
            ids_palabras_tokenizadas
        )
    )
    return new_tags

def tokenizar_y_alinear(dataset):
    """
    Tokeniza las filas del dataset pasado por parámetro, obtiene las NER tags
    correspondientes a cada fila y las alinea con los tokens resultantes de la
    tokenización.

    Parámetros
    ----------
    dataset
        Dataset a tokenizar y alinear
    
    Devuelve
    --------
    inputs_tokenizados
        Dataset pasado por parámetro, ya tokenizado y con las NER tags alineadas
        con los nuevos tokens resultantes

    """

    # Tokenizamos todas las filas del dataset
    inputs_tokenizados = tokenizer(dataset['tokens'], truncation=True, is_split_into_words=True)

    # Obtenemos los IDs de las palabras tokenizadas y las NER tags originales
    ner_tags_originales = dataset['ner_tags']
    nfilas = len(inputs_tokenizados['input_ids'])
    word_ids = list(inputs_tokenizados.word_ids(i) for i in range(nfilas))

    # Alineamos las NER tags originales con los IDs de las palabras tokenizadas
    new_labels = list(map(alinear, word_ids, ner_tags_originales))

    # Creamos una nueva columna en el dataset con las tags alineadas
    inputs_tokenizados['labels'] = new_labels

    return inputs_tokenizados

# Tokenizamos los datos mediante el tokenizer del modelo preentrenado
tokenized_data = data.map(tokenizar_y_alinear, batched=True, remove_columns=data['train'].column_names)


In [ ]:
# --------------------------------------------------------------#
#                        Data collation                         #
#---------------------------------------------------------------#

from transformers import DataCollatorForTokenClassification

# El data collator lo único que hace es hacer todos los inputs del mismo tamaño y
# añadir padding cuando es necesario a cada uno de los features de los datasets.
# Aquí solamente lo invocamos, puesto que más tarde se le pasa al trainer
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
# --------------------------------------------------------------#
#                        Evaluación                             #
#---------------------------------------------------------------#

import numpy as np
import evaluate

metrica = evaluate.load('seqeval')

def evaluar_metricas(predicciones_eval):

    logits, etiquetas = predicciones_eval

    predicciones = np.argmax(logits, axis=-1)

    etiquetas_true = [[id2tag[e] for e in etiqueta if e != -100] for etiqueta in etiquetas]
    predicciones_true = [ [id2tag[p] for p, e in zip(prediccion, etiqueta) if e!=-100] for prediccion, etiqueta in zip(predicciones, etiquetas)]

    metricas = metrica.compute(predictions=predicciones_true, references=etiquetas_true)

    return {
        "precision": metricas['overall_precision'],
        "recall": metricas['overall_recall'],
        "f1": metricas['overall_f1'],
        "accuracy": metricas['overall_accuracy']
    }

In [ ]:
# --------------------------------------------------------------#
#                  Parámetros de entrenamiento                  #
#---------------------------------------------------------------#

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from torch import device

# Cargamos el modelo preentrenado
modelo = AutoModelForTokenClassification.from_pretrained(
    nombre_modelo,
    id2label=id2tag,
    label2id=tag2id
)

# Establecemos los parámetros de entrenamiento
args = TrainingArguments(
    output_dir="finetuned-ner", # Nombre del nuevo modelo
    #eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit = 1,
    #load_best_model_at_end=True,
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01
)

# Invocamos un "entrenador", le pasamos el modelo preentrenado, los
# parámetros de entrenamiento, el dataset, el collator, la
# función de evaluación y el tokenizador
trainer = Trainer(
    model=modelo,
    args=args,
    train_dataset=tokenized_data['train'],
    data_collator=data_collator,
    compute_metrics=evaluar_metricas,
    processing_class=tokenizer
)

In [ ]:
# --------------------------------------------------------------#
#                        Entrenamiento                          #
#---------------------------------------------------------------#

import torch

print('COMENZANDO ENTRENAMIENTO...')

USE_CUDA = True

# Comenzamos a entrenar
trainer.train()

# Guardamos el modelo refinado
torch.save(trainer.model, "./modelo.pt")

In [ ]:
# --------------------------------------------------------------#
#                        Testeo                                 #
#---------------------------------------------------------------#

from transformers import pipeline
from pprint import pp
import torch

frase = "pon una alarma mañana a las diez"

# Debemos cambiarlo al nombre del checkpoint guardado en la carpeta
modelo_checkpoint = "finetuned-ner/checkpoint-22265"

# Cargamos el modelo refinado
clasificador = pipeline(
    "token-classification",
    model=modelo_checkpoint,
    aggregation_strategy="simple"
)

# Inferimos
pp(clasificador(frase))